In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

In [4]:
data = spark.read.csv('dataset/customer_churn.csv',inferSchema=True,header=True)

In [5]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [6]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [8]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [9]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [10]:
assembler = VectorAssembler(inputCols=['Age','Total_Purchase','Account_Manager','Years','Num_Sites'],outputCol='features')

In [12]:
output = assembler.transform(data)

In [13]:
final_data = output.select('features','churn')

In [14]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

In [15]:
from pyspark.ml.classification import LogisticRegression

In [16]:
lr_churn = LogisticRegression(labelCol='churn')

In [17]:
fitted_churn_model = lr_churn.fit(train_churn)

In [19]:
training_sum = fitted_churn_model.summary

In [20]:
training_sum.predictions.describe().show()

+-------+------------------+-------------------+
|summary|             churn|         prediction|
+-------+------------------+-------------------+
|  count|               657|                657|
|   mean|0.1719939117199391| 0.1339421613394216|
| stddev|0.3776626769007658|0.34084965975049586|
|    min|               0.0|                0.0|
|    max|               1.0|                1.0|
+-------+------------------+-------------------+



In [21]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [22]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [23]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[28.0,9090.43,1.0...|    0|[1.54428872431150...|[0.82408731488278...|       0.0|
|[28.0,11204.23,0....|    0|[1.67285580240271...|[0.84195620243718...|       0.0|
|[29.0,9617.59,0.0...|    0|[4.52633881547338...|[0.98929560542874...|       0.0|
|[29.0,11274.46,1....|    0|[4.41871215459755...|[0.98809372700056...|       0.0|
|[30.0,8874.83,0.0...|    0|[3.25590989819091...|[0.96288489681065...|       0.0|
|[30.0,12788.37,0....|    0|[2.33995085138702...|[0.91213214613197...|       0.0|
|[31.0,9574.89,0.0...|    0|[3.49259505411359...|[0.97047634019410...|       0.0|
|[31.0,11297.57,1....|    1|[0.94412692075012...|[0.71993252191303...|       0.0|
|[32.0,8011.38,0.0...|    0|[2.06971988211790...|[0.88792508878589...|       0.0|
|[32.0,11715.72,

In [24]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='churn')

In [25]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [26]:
auc

0.7740750459197061

In [27]:
# predict on new data

final_lr_model = lr_churn.fit(final_data)

In [30]:
new_customers = spark.read.csv('dataset/new_customers.csv',inferSchema=True,header=True)

In [31]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: integer (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [32]:
test_new_customers = assembler.transform(new_customers)

In [33]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: integer (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [34]:
final_results = final_lr_model.transform(test_new_customers)

In [36]:
final_results.select('company','prediction').show()

+--------------------+----------+
|             company|prediction|
+--------------------+----------+
|          Harvey LLC|       0.0|
|          Wilson PLC|       1.0|
|Miller, Johnson a...|       1.0|
|           Smith Inc|       0.0|
|          Love-Jones|       0.0|
|        Kelly-Warren|       0.0|
|   Reynolds-Sheppard|       1.0|
|          Singh-Cole|       0.0|
|           Lopez PLC|       1.0|
|       Reed-Martinez|       1.0|
|Briggs, Lamb and ...|       0.0|
|    Figueroa-Maynard|       1.0|
|     Abbott-Thompson|       1.0|
|Smith, Kim and Ma...|       1.0|
|Snyder, Lee and M...|       0.0|
|      Sanders-Pierce|       1.0|
|Andrews, Adams an...|       1.0|
|Morgan, Phillips ...|       1.0|
|      Villanueva LLC|       0.0|
|Berry, Orr and Ca...|       0.0|
+--------------------+----------+
only showing top 20 rows

